In [1]:
from agents import Agent, Runner, TResponseInputItem

simple_agent = Agent(
    name="Bob",
    instructions="You are a friendly assistant. Reply concisely."
)

convo: list[TResponseInputItem] = [
    {"content": "Hey!", "role": "user"},
    {"content": "Hey! How are you? How can I help you today?", "role": "assistant"},
    {"content": "Im having a bad day", "role": "user"}
]
result = await Runner.run(simple_agent, convo)
for message in result.to_input_list():
    print(message)

{'content': 'Hey!', 'role': 'user'}
{'content': 'Hey! How are you? How can I help you today?', 'role': 'assistant'}
{'content': 'Im having a bad day', 'role': 'user'}
{'id': 'msg_685954bbfbd081988729b37bd6a50e6f082fd5af570a2a21', 'content': [{'annotations': [], 'text': "I'm sorry to hear that. Do you want to talk about what's going on?", 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}


In [3]:
convo: list[TResponseInputItem] = []
print("You are now chatting with agent Bob. Type 'exit' to end the conversation.")
while True:
    user_input = input("You: ")
    print("You: " + user_input)

    if user_input == "exit":
        print("Goodbye!")
        break

    convo.append({"content": user_input, "role": "user"})
    result = await Runner.run(simple_agent, convo)

    print(f"Agent: {result.final_output}")

    convo = result.to_input_list()


You are now chatting with agent Bob. Type 'exit' to end the conversation.
You: hi
Agent: Hello! How can I assist you today?
You: no
Agent: Alright! If you change your mind or need anything, feel free to let me know.
You: exit
Goodbye!


In [7]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples"
)

def on_math_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to math tutor agent")

# This agent has the capability to handoff to either the history or math tutor agent
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question." +
    "If neither agent is relevant, provide a general response.",
    handoffs=[history_tutor_agent, handoff(math_tutor_agent, on_handoff=on_math_handoff)]
)

convo: list[TResponseInputItem] = []
last_agent = triage_agent
print("You are now chatting with the triage agent. Type 'exit' to end the conversation.")
while True:
    user_input = input("You: ")
    print("You: " + user_input)

    if user_input == "exit":
        print("Goodbye!")
        break

    print(f"{last_agent.name}: {result.final_output}\n")
    convo.append({"content": user_input, "role": "user"})
    result = await Runner.run(last_agent, convo)

    convo = result.to_input_list()
    last_agent = result.last_agent


You are now chatting with the triage agent. Type 'exit' to end the conversation.
You: hi
Triage Agent: Of course! What do you need help with? Is it related to a specific subject or topic?

You: 
Triage Agent: Hello! How can I assist you today?

You: 
Triage Agent: What would you like to talk about or need help with?

You: 
Triage Agent: It seems like your message might not have gone through. Could you please try sending it again?

You: exit
Goodbye!
